In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import casadi as ca
from decision_vars import DecisionVarSet

In [2]:
# Test decision vars
from decision_vars import DecisionVarSet
dvs = DecisionVarSet(attr_names = ['lb'], name = 'sneaker/')
dvs.add_vars({'x':[1.0, 2.0]}, lb={'x':[0, 0]})
print(dvs)
dvs2 = DecisionVarSet(attr_names = ['lb'])
dvs2.add_vars(inits={'x':2.0}, lb={'y':3.0})
dvs += dvs2
print(dvs.vectorize('lb'))
d2 = dvs.dictize(np.array([0.1, 0.3, 0.5]))
print(d2)
d = {'test':1}
d.update(dvs2)
print(d)

***** Decision Vars *****
Attributes: ['sym', 'init', 'lb']
Vars: 
  sneaker/x: [x_0, x_1], shape: (2, 1)

[0, 0, -inf]
{'sneaker/x': array([0.1, 0.3]), 'x': array(0.5)}
{'test': 1, 'x': SX(x)}


In [3]:
from robot import *
rob = Robot('config/franka.urdf')

Building robot model from config/franka.urdf with TCP fr3_link8


In [32]:
from contact import Contact
pars = {'pos': [0, 0, 0],
        'stiff': [0, 10, 10],
        'rest':[0, 0, 0.0]}
c = Contact('friendo', pars)
q = ca.SX.sym('q',7)
p, R = rob.fwd_kin(q)
print(c.get_force({'p':ca.DM([0.5, 0.3, -0.5]), 'R':ca.DM.ones(3)}))

c2 = Contact('c2',pars, ['stiff'], {'cov':{}, 'noise':{}})
print(c2.get_force({'p':ca.DM([0.5, 0.3, -0.5]), 
                    'R':ca.DM.ones(3), 
                    'c2/stiff':ca.DM([10, 0, 0])}))

[0, -3, 5]
[-0, -0, 0]


In [77]:
# Test convergence
from robot import Robot
par1 = {'pos': [0, 0, 0.0],
        'stiff': [0, 0, 0],
        'rest':[0, 0, 0.2]}
par2 = {'pos': [0, 0, 0],
        'stiff': [0, 0, 0],
        'rest': [0, 0.5, 0]}
c1 = Contact('c1/', par1, ['stiff'], {'cov':{}, 'noise':{}})
c2 = Contact('c2/', par2, [], {'cov':{}, 'noise':{}})

rob = Robot('config/franka.urdf', subsys = [c1, c2])
step = rob.build_disc_dyn(0.1)
step_core = rob.disc_dyn_core
# test that it converges as expected 
q = np.zeros(7)
dq = np.zeros(7)
tau = np.zeros(7)
state = {'q':q, 'dq':dq, 'tau_input':tau}
state['c1/stiff'] = ca.DM([0, 0, 1000])
%timeit step

for _ in range(100):
    res = step(**state)
    state['q']  = res['q_next']
    state['dq'] = res['dq_next']
res_dict = rob.get_statedict(state)
print(res_dict['p'])
#print(rob.inv_mass_fn(np.zeros(7)))

print(res_dict['c1/F'])
print(res_dict['c2/F'])

Building robot model from config/franka.urdf with TCP fr3_link8
  with subsys ['c1/', 'c2/']
[0.565695, 0.0102108, 0.2]
[-0, -0, 2.66454e-12]
[0, 0, 0]


In [46]:
# Check that the spawner works
from helper_fns import spawn_models
models = spawn_models('config/franka.yaml', 'config/attrs.yaml', 'config/contact.yaml', sym_vars = ['pos'])
for name, model in models.items():
    step = model.build_disc_dyn(step_size=0.1)
    n_in = step.size_in(0)
    res = step(np.zeros(n_in), np.zeros(7))
    res_dict = model.get_statedict(res
    print(f"{name} has step fn of {step}")
    print(f" which evals to {res_dict}")


SyntaxError: invalid syntax (2145105069.py, line 9)